<a href="https://colab.research.google.com/github/sibiraja/cs2241project/blob/main/cs2241.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages and clone the arithmetic coding repository
!pip install torch transformers datasets
!git clone https://github.com/nayuki/Reference-arithmetic-coding.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import sys
import io
import math
import torch
import torch.nn.functional as F
import gzip
import numpy as np
from itertools import islice
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

# Append the arithmetic coding repo's python folder to sys.path so we can import it.
sys.path.append("Reference-arithmetic-coding/python")
from arithmeticcoding import ArithmeticEncoder, ArithmeticDecoder, SimpleFrequencyTable, BitOutputStream, BitInputStream

In [ ]:
print("GPU available:", torch.cuda.is_available())

GPU available: True


TODO: after pipeline works on just 1 piece of text, split data into training and test, and use test set for analysis (training is for LLM fine-tuning)

In [ ]:
#############################################
# 1. Load a small subset of the Casehold dataset
#############################################
ds = load_dataset("casehold/casehold", "all")
# Use a small subset for testing – here we select the first 100 examples.
if "train" in ds:
    small_ds = ds["train"].select(range(100))
else:
    small_ds = ds.select(range(100))

print("Dataset columns:", small_ds.column_names)
# We will use the 'citing_prompt' column as the text input.
sample_text = small_ds[0]["citing_prompt"]
print("Sample text:\n", sample_text)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


casehold.py:   0%|          | 0.00/8.68k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/46.2M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/5.79M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/5.81M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/42509 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5314 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5314 [00:00<?, ? examples/s]

Dataset columns: ['example_id', 'citing_prompt', 'holding_0', 'holding_1', 'holding_2', 'holding_3', 'holding_4', 'label']
Sample text:
 Drapeau’s cohorts, the cohort would be a “victim” of making the bomb. Further, firebombs are inherently dangerous. There is no peaceful purpose for making a bomb. Felony offenses that involve explosives qualify as “violent crimes” for purposes of enhancing the sentences of career offenders. See 18 U.S.C. § 924(e)(2)(B)(ii) (defining a “violent felony” as: “any crime punishable by imprisonment for a term exceeding one year ... that ... involves use of explosives”). Courts have found possession of a'bomb to be a crime of violence based on the lack of a nonviolent purpose for a bomb and the fact that, by its very nature, there is a substantial risk that the bomb would be used against the person or property of another. See United States v. Newman, 125 F.3d 863 (10th Cir.1997) (unpublished) (<HOLDING>); United States v. Dodge, 846 F.Supp. 181,


In [ ]:
#############################################
# 2. Load GPT-2 (non-finetuned) pipeline
#############################################
gpt2_tokenizer = AutoTokenizer.from_pretrained("gpt2")
gpt2_model = AutoModelForCausalLM.from_pretrained("gpt2")
gpt2_model.eval()  # Set to evaluation mode

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
#############################################
# 3. Load neoGPT (non-finetuned) pipeline
#############################################
neoGPT_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125m")
neoGPT_model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125m")
neoGPT_model.eval()

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPTNeoBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=False)
            (q_proj): Linear(in_features=768, out_features=768, bias=False)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_fe

#TODO: FINETUNE LLMS

In [ ]:
#############################################
# 4. Placeholder for Fine-Tuning Code
#############################################
# TODO: Insert fine-tuning code here to specialize the LLMs on domain-specific data.
# For example:
# def fine_tune_model(model, tokenizer, dataset):
#     # Set up TrainingArguments, Trainer, etc.
#     # trainer = Trainer(model=model, args=training_args, train_dataset=dataset, ...)
#     # trainer.train()
#     pass

In [ ]:
#############################################
# 5. Implement Arithmetic Coding Functions Using the Cloned Repo
#############################################
def create_frequency_table(probs_np, total=4096):
    """
    Given a numpy array of probabilities (summing to ~1),
    create a list of integer frequencies that sum exactly to 'total'.
    Frequencies are adjusted deterministically.
    """
    # Initially, compute frequencies by flooring the product.
    freqs = np.floor(probs_np * total).astype(int)
    # Ensure each frequency is at least 1.
    freqs = np.maximum(freqs, 1)
    current_total = freqs.sum()
    diff = total - current_total
    # Adjust frequencies: if diff > 0, add 1 to indices with highest probabilities;
    # if diff < 0, subtract 1 from indices with lowest probabilities (ensuring frequency remains >= 1).
    if diff > 0:
        indices = np.argsort(-probs_np)  # descending order
        for idx in indices:
            if diff <= 0:
                break
            freqs[idx] += 1
            diff -= 1
    elif diff < 0:
        indices = np.argsort(probs_np)  # ascending order
        for idx in indices:
            if diff >= 0:
                break
            if freqs[idx] > 1:
                freqs[idx] -= 1
                diff += 1
    return list(freqs)

def arithmetic_encode_token(probs, token_id, total=4096):
    """
    Encodes a single token using arithmetic coding.
    'probs' is a tensor of predicted probabilities (after softmax) for the next token.
    Returns the encoded bytes.
    """
    probs_np = probs.cpu().numpy()  # Convert to numpy array.
    freq_list = create_frequency_table(probs_np, total=total)
    print("Encoding: Token ID =", token_id, "First 10 frequencies =", freq_list[:10], "Sum =", sum(freq_list))
    freq_table = SimpleFrequencyTable(freq_list)

    output = io.BytesIO()
    bitout = BitOutputStream(output)
    encoder = ArithmeticEncoder(32, bitout)
    # Write the symbol using the frequency table.
    encoder.write(freq_table, token_id)
    encoder.finish()
    encoded_bytes = output.getvalue()
    return encoded_bytes

def arithmetic_decode_token(encoded_bytes, probs, total=4096):
    """
    Decodes a single token from the given encoded bytes using the predicted probability distribution.
    Returns the decoded token id.
    """
    probs_np = probs.cpu().numpy()
    freq_list = create_frequency_table(probs_np, total=total)
    print("Decoding: Frequencies =", freq_list[:10], "Sum =", sum(freq_list))
    freq_table = SimpleFrequencyTable(freq_list)

    input_stream = io.BytesIO(encoded_bytes)
    bitin = BitInputStream(input_stream)
    decoder = ArithmeticDecoder(32, bitin)
    token_id = decoder.read(freq_table)
    return token_id

#=== Testing CREATE FREQ TABLE FUNCTION ====

In [ ]:
# Create a fixed probability vector (sums to 1)
fixed_probs = np.array([0.1, 0.2, 0.3, 0.4])
total = 1000  # Use a smaller total for clarity

# Run the frequency table creation multiple times to ensure consistency
for i in range(5):
    freq_table = create_frequency_table(fixed_probs, total=total)
    print(f"Run {i}: Frequency table = {freq_table}, Sum = {sum(freq_table)}")

Run 0: Frequency table = [np.int64(100), np.int64(200), np.int64(300), np.int64(400)], Sum = 1000
Run 1: Frequency table = [np.int64(100), np.int64(200), np.int64(300), np.int64(400)], Sum = 1000
Run 2: Frequency table = [np.int64(100), np.int64(200), np.int64(300), np.int64(400)], Sum = 1000
Run 3: Frequency table = [np.int64(100), np.int64(200), np.int64(300), np.int64(400)], Sum = 1000
Run 4: Frequency table = [np.int64(100), np.int64(200), np.int64(300), np.int64(400)], Sum = 1000


#====DONE TESTING =====

In [ ]:
#############################################
# 6. Update the Encode/Decode Pipeline Functions (starting from the first token)
#############################################
def encode_text(text, model, tokenizer):
    """
    Encode the text token-by-token using the provided model and actual arithmetic coding.
    Uses a BOS token (or EOS as a fallback) as the initial context.
    Returns a list of encoded byte strings.
    """
    tokens = tokenizer(text, return_tensors="pt")
    input_ids = tokens.input_ids  # shape: (1, seq_len)
    encoded_stream = []

    # Define starting context using BOS token if available; otherwise, fallback to EOS.
    bos_id = tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.eos_token_id
    bos_context = torch.tensor([[bos_id]])

    # Loop over all tokens in the sequence.
    for i in range(input_ids.shape[1]):
        current_context = bos_context if i == 0 else input_ids[:, :i]
        with torch.no_grad():
            outputs = model(current_context)
        logits = outputs.logits[0, -1, :]
        probs = F.softmax(logits, dim=-1)
        token_id = input_ids[0, i].item()

        # Encode the token using arithmetic coding.
        encoded_token = arithmetic_encode_token(probs, token_id)
        encoded_stream.append(encoded_token)

    return encoded_stream

def decode_text(encoded_stream, model, tokenizer):
    """
    Decode the encoded stream back into text.
    Initializes with the same BOS token used during encoding.
    """
    bos_id = tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.eos_token_id
    decoded_ids = [bos_id]

    # For each encoded token, use the model to obtain the probability distribution for decoding.
    # (In a real scenario, the decoder would use context; here we mimic the process token-by-token.)
    for encoded_token in encoded_stream:
        # Use the current decoded tokens as context.
        context_ids = torch.tensor([decoded_ids])
        with torch.no_grad():
            outputs = model(context_ids)
        logits = outputs.logits[0, -1, :]
        probs = F.softmax(logits, dim=-1)
        token_id = arithmetic_decode_token(encoded_token, probs)
        decoded_ids.append(token_id)

    decoded_text = tokenizer.decode(decoded_ids, skip_special_tokens=True)
    return decoded_text



In [ ]:
# # -------------------------------------------------
# # Simulated Arithmetic Coding Functions (Placeholders)
# # -------------------------------------------------
# def arithmetic_encode(prob_dist, token_id):
#     """
#     Placeholder: In a real implementation, use an arithmetic coding library.
#     Here we simulate encoding by returning a string representation of the token_id.
#     """
#     return f"encoded({token_id})"

# def arithmetic_decode(encoded_token, prob_dist=None):
#     """
#     Placeholder: Simulate decoding by extracting the token_id from the string.
#     This simplistic approach assumes the string format is 'encoded(token_id)'.
#     """
#     token_id_str = encoded_token[len("encoded("):-1]
#     return int(token_id_str)

#   # -------------------------------------------------
# # Encode and Decode Pipeline Functions
# # -------------------------------------------------
# def encode_text(text, model, tokenizer):
#     """
#     Encode the text token-by-token using GPT-2, starting with a dummy BOS context.
#     """
#     tokens = tokenizer(text, return_tensors="pt")
#     input_ids = tokens.input_ids  # shape: (1, seq_len)
#     encoded_stream = []

#     # Define a starting context using a BOS token.
#     # If the tokenizer doesn't have a BOS token, use its EOS token as a fallback.
#     bos_id = tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.eos_token_id
#     bos_context = torch.tensor([[bos_id]])

#     # Loop over all tokens in the input.
#     for i in range(input_ids.shape[1]):
#         # For the first token, use the bos_context; for others, use all tokens before i.
#         current_context = bos_context if i == 0 else input_ids[:, :i]
#         with torch.no_grad():
#             outputs = model(current_context)
#         logits = outputs.logits[0, -1, :]
#         probs = torch.nn.functional.softmax(logits, dim=-1)
#         token_id = input_ids[0, i].item()

#         # Simulate encoding using the placeholder arithmetic function.
#         encoded_token = arithmetic_encode(probs, token_id)
#         encoded_stream.append(encoded_token)

#     return encoded_stream

# def decode_text(encoded_stream, model, tokenizer, initial_context=""):
#     """
#     Decode the encoded stream back into text.
#     We initialize with the same BOS token used during encoding.
#     """
#     # Use the same starting context as in encoding.
#     bos_id = tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.eos_token_id
#     decoded_ids = [bos_id]  # Start with the BOS token.

#     # For each encoded token, simulate decoding.
#     for encoded_token in encoded_stream:
#         token_id = arithmetic_decode(encoded_token)
#         decoded_ids.append(token_id)

#     # Decode the full sequence, skipping special tokens if needed.
#     decoded_text = tokenizer.decode(decoded_ids, skip_special_tokens=True)
#     return decoded_text


In [ ]:

#############################################
# 7. Define Baseline Compression Functions
#############################################
# GZIP Compression
def compress_gzip(text):
    compressed = gzip.compress(text.encode('utf-8'))
    return compressed

# LZW Compression Implementation
def lzw_compress(uncompressed):
    uncompressed_bytes = uncompressed.encode('utf-8')
    dict_size = 256
    dictionary = {bytes([i]): i for i in range(dict_size)}
    w = b""
    result = []
    for b in uncompressed_bytes:
        c = bytes([b])
        wc = w + c
        if wc in dictionary:
            w = wc
        else:
            result.append(dictionary[w])
            dictionary[wc] = dict_size
            dict_size += 1
            w = c
    if w:
        result.append(dictionary[w])
    return result

def lzw_compression_ratio(text):
    original_size = len(text.encode('utf-8'))  # in bytes
    compressed_codes = lzw_compress(text)
    max_code = max(compressed_codes) if compressed_codes else 0
    bits_per_code = math.ceil(math.log2(max_code + 1)) if max_code > 0 else 0
    compressed_size_bits = len(compressed_codes) * bits_per_code
    compressed_size_bytes = compressed_size_bits / 8
    ratio = compressed_size_bytes / original_size if original_size > 0 else 0
    return ratio, len(compressed_codes)


In [ ]:
#############################################
# 8. Run the pipeline for GPT-2 and neoGPT on the sample text
#############################################
print("\n--- GPT-2 Pipeline ---")
encoded_gpt2 = encode_text(sample_text, gpt2_model, gpt2_tokenizer)
decoded_gpt2 = decode_text(encoded_gpt2, gpt2_model, gpt2_tokenizer)
print("Decoded text (GPT-2):\n", decoded_gpt2)

print("\n--- neoGPT Pipeline ---")
encoded_neoGPT = encode_text(sample_text, neoGPT_model, neoGPT_tokenizer)
decoded_neoGPT = decode_text(encoded_neoGPT, neoGPT_model, neoGPT_tokenizer)
print("Decoded text (neoGPT):\n", decoded_neoGPT)


--- GPT-2 Pipeline ---
Encoding: Token ID = 35 First 10 frequencies = [np.int64(1), np.int64(97), np.int64(2), np.int64(3), np.int64(1), np.int64(1), np.int64(23), np.int64(22), np.int64(5), np.int64(6)] Sum = 52032
Encoding: Token ID = 13484 First 10 frequencies = [np.int64(4), np.int64(9), np.int64(1), np.int64(1), np.int64(2), np.int64(1), np.int64(10), np.int64(6), np.int64(29), np.int64(1)] Sum = 51637
Encoding: Token ID = 559 First 10 frequencies = [np.int64(3), np.int64(18), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(5), np.int64(1), np.int64(9), np.int64(1)] Sum = 52189
Encoding: Token ID = 447 First 10 frequencies = [np.int64(1), np.int64(7), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(2), np.int64(1), np.int64(44), np.int64(1)] Sum = 52692
Encoding: Token ID = 247 First 10 frequencies = [np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1)] Sum = 54207
Encoding: Tok

In [ ]:
#############################################
# 9. Evaluate Baseline Compression Methods
#############################################
original_size = len(sample_text.encode('utf-8'))
print("\nOriginal size (bytes):", original_size)

gzip_compressed = compress_gzip(sample_text)
print("GZIP compressed size (bytes):", len(gzip_compressed))

lzw_ratio, lzw_codes_len = lzw_compression_ratio(sample_text)
print("LZW compressed code count:", lzw_codes_len)
print("LZW approximate compressed size ratio (relative to original):", lzw_ratio)


Original size (bytes): 869
GZIP compressed size (bytes): 533
LZW compressed code count: 527
LZW approximate compressed size ratio (relative to original): 0.7580552359033371


In [ ]:
#############################################
# 10. Placeholder: Future Steps
#############################################
# TODO: Evaluate bits-per-character and compression ratios of the LLM-based arithmetic coding
# compressor vs. gzip and LZW.
# Experiment with fine-tuning the LLMs on domain-specific data to potentially improve compression.

THE ARITHMETIC ENCODING CODE IS NOT WORKING CORRECTLY BECAUSE NOW EVEN GPT IS ENCODING/DECODING CORRECTLY

In [ ]:
import numpy as np
import io

# Use the total resolution and k as parameters.
def create_frequency_table_topk(probs_np, total=4096, k=100):
    """
    Given a full probability vector (for the entire vocabulary),
    select the top-k tokens (indices) and compute integer frequencies
    (summing exactly to 'total') for those tokens.

    Returns:
      topk_indices: a numpy array of the indices of the top-k tokens
      freqs: a list of integer frequencies (length k)
    """
    # Get indices of the top-k tokens (in descending order by probability)
    topk_indices = np.argsort(-probs_np)[:k]
    topk_probs = probs_np[topk_indices]
    # Renormalize so that the top-k probabilities sum to 1.
    topk_probs = topk_probs / topk_probs.sum()

    # Compute frequencies by flooring the product.
    freqs = np.floor(topk_probs * total).astype(int)
    # Ensure that each token gets at least 1.
    freqs = np.maximum(freqs, 1)
    current_total = freqs.sum()
    diff = total - current_total

    # Adjust frequencies to match the total exactly.
    if diff > 0:
        indices = np.argsort(-topk_probs)  # highest probabilities first
        for idx in indices:
            if diff <= 0:
                break
            freqs[idx] += 1
            diff -= 1
    elif diff < 0:
        indices = np.argsort(topk_probs)  # lowest probabilities first
        for idx in indices:
            if diff >= 0:
                break
            if freqs[idx] > 1:
                freqs[idx] -= 1
                diff += 1
    return topk_indices, freqs.tolist()

def arithmetic_encode_token_topk(probs, token_id, total=4096, k=100):
    """
    Encodes a single token using arithmetic coding,
    restricting the probability distribution to the top-k tokens.

    If the token is not in the top-k, an error is raised.
    Returns the encoded bytes and prints debugging info.
    """
    probs_np = probs.cpu().numpy()
    topk_indices, freqs = create_frequency_table_topk(probs_np, total=total, k=k)
    print("Encoding: Top-k indices:", topk_indices)
    print("Encoding: Frequencies (first 10):", freqs[:10], "Sum =", sum(freqs))

    # Check if the target token is in the top-k set.
    if token_id not in topk_indices:
        print("Token id", token_id, "not in top-k set; cannot encode.")
        raise ValueError("Token not in top-k set.")

    # Map the token id to its new index in the top-k list.
    new_token_id = int(np.where(topk_indices == token_id)[0][0])

    # Build the frequency table using the library's SimpleFrequencyTable.
    freq_table = SimpleFrequencyTable(freqs)
    output = io.BytesIO()
    bitout = BitOutputStream(output)
    encoder = ArithmeticEncoder(32, bitout)
    encoder.write(freq_table, new_token_id)
    encoder.finish()
    encoded_bytes = output.getvalue()
    return encoded_bytes, topk_indices

def arithmetic_decode_token_topk(encoded_bytes, probs, total=4096, k=100):
    """
    Decodes a single token using arithmetic coding,
    restricting the probability distribution to the top-k tokens.
    Returns the original token id.
    """
    probs_np = probs.cpu().numpy()
    topk_indices, freqs = create_frequency_table_topk(probs_np, total=total, k=k)
    print("Decoding: Top-k indices:", topk_indices)
    print("Decoding: Frequencies (first 10):", freqs[:10], "Sum =", sum(freqs))

    freq_table = SimpleFrequencyTable(freqs)
    input_stream = io.BytesIO(encoded_bytes)
    bitin = BitInputStream(input_stream)
    decoder = ArithmeticDecoder(32, bitin)
    new_token_id = decoder.read(freq_table)
    # Map back from the top-k index to the original token id.
    original_token_id = int(topk_indices[new_token_id])
    return original_token_id


In [ ]:
# Choose a sample text and tokenize it.
sample_text = "This is a test."
tokens = gpt2_tokenizer(sample_text, return_tensors="pt")
input_ids = tokens.input_ids  # shape: (1, seq_len)

# Let's choose a token index to test. For example, i = 2.
i = 2

original_token_id = input_ids[0, i].item()
print("Original token id:", original_token_id, "Token:", gpt2_tokenizer.decode([original_token_id]))

bos_id = gpt2_tokenizer.bos_token_id if gpt2_tokenizer.bos_token_id is not None else gpt2_tokenizer.eos_token_id
current_context = torch.tensor([[bos_id]]) if i == 0 else input_ids[:, :i]

with torch.no_grad():
    outputs = gpt2_model(current_context)
logits = outputs.logits[0, -1, :]
probs = torch.nn.functional.softmax(logits, dim=-1)

# Encode using the top-k function (e.g. top 100 tokens).
encoded_bytes, topk_indices = arithmetic_encode_token_topk(probs, original_token_id, total=4096, k=100)
decoded_token_id = arithmetic_decode_token_topk(encoded_bytes, probs, total=4096, k=100)

print("Decoded token id:", decoded_token_id, "Token:", gpt2_tokenizer.decode([decoded_token_id]))


Original token id: 257 Token:  a
Encoding: Top-k indices: [  257   262   407   644  1521   281   703   530   810   655   616   780
  1223   635   477   845  1107  2192   674  1016  1593  3446  1682   645
  2081    11   546   691   618   287  2407  1194   523   588   534   284
  2391  4753  1049   636   991   517   329  3729   922  5543   416  2495
  2592   340  6189   884   355  1290 10582  1884   465  5600  2147  2233
  4988  1444   511  4084   783  3573   286   881  3499   422  4457  6209
  2048  2092  2089  4998   772   326  6986  5626  5836  1912   826  1180
  3737  1690   617  3190  1611   319  1865  1744   379  1464  1760  1165
  6635   749  1541  3221]
Encoding: Frequencies (first 10): [788, 643, 328, 234, 192, 187, 108, 100, 96, 76] Sum = 4096
Decoding: Top-k indices: [  257   262   407   644  1521   281   703   530   810   655   616   780
  1223   635   477   845  1107  2192   674  1016  1593  3446  1682   645
  2081    11   546   691   618   287  2407  1194   523   588   534 

In [ ]:
#############################################
# 6. Updated Encode/Decode Pipeline Functions (using top-k arithmetic coding)
#############################################
def encode_text_topk(text, model, tokenizer, total=4096, k=100):
    """
    Encode the text token-by-token using the provided model and top-k arithmetic coding.
    Uses a BOS token (or EOS as fallback) as the initial context.
    Returns a list of encoded byte strings.
    """
    tokens = tokenizer(text, return_tensors="pt")
    input_ids = tokens.input_ids  # shape: (1, seq_len)
    encoded_stream = []

    # Use BOS (or EOS if BOS isn't defined) as starting context.
    bos_id = tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.eos_token_id
    bos_context = torch.tensor([[bos_id]])

    # Loop over all tokens in the sequence.
    for i in range(input_ids.shape[1]):
        current_context = bos_context if i == 0 else input_ids[:, :i]
        with torch.no_grad():
            outputs = model(current_context)
        logits = outputs.logits[0, -1, :]
        probs = torch.nn.functional.softmax(logits, dim=-1)
        token_id = input_ids[0, i].item()

        # Use the top-k encoding function.
        encoded_token, _ = arithmetic_encode_token_topk(probs, token_id, total=total, k=k)
        encoded_stream.append(encoded_token)

    return encoded_stream

def decode_text_topk(encoded_stream, model, tokenizer, total=4096, k=100):
    """
    Decode the encoded stream back into text using the top-k arithmetic coding.
    Initializes with the same BOS token as used during encoding.
    """
    bos_id = tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.eos_token_id
    decoded_ids = [bos_id]

    # For each encoded token, get the context from the tokens decoded so far.
    for encoded_token in encoded_stream:
        context_ids = torch.tensor([decoded_ids])
        with torch.no_grad():
            outputs = model(context_ids)
        logits = outputs.logits[0, -1, :]
        probs = torch.nn.functional.softmax(logits, dim=-1)
        token_id = arithmetic_decode_token_topk(encoded_token, probs, total=total, k=k)
        decoded_ids.append(token_id)

    decoded_text = tokenizer.decode(decoded_ids, skip_special_tokens=True)
    return decoded_text

In [ ]:
#############################################
# 7. Baseline Compression Functions (unchanged)
#############################################
# GZIP Compression
def compress_gzip(text):
    compressed = gzip.compress(text.encode('utf-8'))
    return compressed

# LZW Compression Implementation
def lzw_compress(uncompressed):
    uncompressed_bytes = uncompressed.encode('utf-8')
    dict_size = 256
    dictionary = {bytes([i]): i for i in range(dict_size)}
    w = b""
    result = []
    for b in uncompressed_bytes:
        c = bytes([b])
        wc = w + c
        if wc in dictionary:
            w = wc
        else:
            result.append(dictionary[w])
            dictionary[wc] = dict_size
            dict_size += 1
            w = c
    if w:
        result.append(dictionary[w])
    return result

def lzw_compression_ratio(text):
    original_size = len(text.encode('utf-8'))  # in bytes
    compressed_codes = lzw_compress(text)
    max_code = max(compressed_codes) if compressed_codes else 0
    bits_per_code = math.ceil(math.log2(max_code + 1)) if max_code > 0 else 0
    compressed_size_bits = len(compressed_codes) * bits_per_code
    compressed_size_bytes = compressed_size_bits / 8
    ratio = compressed_size_bytes / original_size if original_size > 0 else 0
    return ratio, len(compressed_codes)

In [ ]:
#############################################
# 8. Run the pipeline for GPT-2 and neoGPT on the sample text
#############################################
# Assume sample_text is defined (e.g., from your dataset)
sample_text = "How are you"
print("\n--- GPT-2 Pipeline (top-k) ---")
encoded_gpt2 = encode_text_topk(sample_text, gpt2_model, gpt2_tokenizer, total=4096, k=500)
decoded_gpt2 = decode_text_topk(encoded_gpt2, gpt2_model, gpt2_tokenizer, total=4096, k=500)
print("Decoded text (GPT-2):\n", decoded_gpt2)

# For neoGPT, assume neoGPT_model and neoGPT_tokenizer are defined
print("\n--- neoGPT Pipeline (top-k) ---")
encoded_neoGPT = encode_text_topk(sample_text, neoGPT_model, neoGPT_tokenizer, total=4096, k=500)
decoded_neoGPT = decode_text_topk(encoded_neoGPT, neoGPT_model, neoGPT_tokenizer, total=4096, k=500)
print("Decoded text (neoGPT):\n", decoded_neoGPT)


--- GPT-2 Pipeline (top-k) ---
Encoding: Top-k indices: [  198   464     1    32    40   818    13  1026    50  1212    33    12
    34  1135    16    51     6    47     7    38    35    39    44  1639
    37    42  1532    17    25    49    43    52  1890  1858    45    36
  1722    54    11    41  2061  2514    58    18  2215  2202    53    19
    14  3198  3886  1544    15  1537    20    46  2953  2396  3152  3666
  2437  2025    82  1870 50256    64  3237     9    22    62    23   259
  2504     8  3260    29    55    57  2949   940    56  2990  4863  1169
    24  1525    21   290    72  3844  3791  4342  3633  3347   986  1925
  1273  3673     3  5159   357    31    27    65  1065  2484  5122    79
    48    10  4366  5492   960  5211  1157  5956    66     2  5189   271
  1399   628  3792    71  2937    77  3041  7120  2348  4711  5195 11649
    30    83  8585   392  4821  6214  1314    73  1129  3855  1415  5167
  5703  3109  7571   286  9590  6191  3118 11605  1433   785  1462 

In [ ]:
#############################################
# 9. Evaluate Baseline Compression Methods
#############################################
original_size = len(sample_text.encode('utf-8'))
print("\nOriginal size (bytes):", original_size)

gzip_compressed = compress_gzip(sample_text)
print("GZIP compressed size (bytes):", len(gzip_compressed))

lzw_ratio, lzw_codes_len = lzw_compression_ratio(sample_text)
print("LZW compressed code count:", lzw_codes_len)
print("LZW approximate compressed size ratio (relative to original):", lzw_ratio)


Original size (bytes): 15
GZIP compressed size (bytes): 33
LZW compressed code count: 14
LZW approximate compressed size ratio (relative to original): 1.05


what's happening is that the sample text:


> Drapeau’s cohorts, the cohort would be a “victim” of making the bomb. Further, firebombs are inherently dangerous. There is no peaceful purpose for making a bomb. Felony offenses that involve explosives qualify as “violent crimes” for purposes of enhancing the sentences of career offenders. See 18 U.S.C. § 924(e)(2)(B)(ii) (defining a “violent felony” as: “any crime punishable by imprisonment for a term exceeding one year ... that ... involves use of explosives”). Courts have found possession of a'bomb to be a crime of violence based on the lack of a nonviolent purpose for a bomb and the fact that, by its very nature, there is a substantial risk that the bomb would be used against the person or property of another. See United States v. Newman, 125 F.3d 863 (10th Cir.1997) (unpublished) (<HOLDING>); United States v. Dodge, 846 F.Supp. 181,



starts off with "Drapeau’s", which is a token not in the top-k vocabulary (or any vocabulary) for GPT-2.

I think I need to look into easier texts to try out. perhaps a much more simple english dataset

In [ ]:
!pip install -U bitsandbytes
!pip install -U transformers accelerate

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineG

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "mistralai/Mistral-7B-Instruct-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
import torch
from collections import Counter

def encode_text_topk(text, model, tokenizer, total=4096, k=200):
    input_ids = tokenizer(text, return_tensors="pt").input_ids.to(model.device)
    with torch.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits[0]

    # Flatten logits, count top-k most common token indices
    probs = torch.softmax(logits, dim=-1)
    token_counts = torch.sum(probs, dim=0).cpu()
    topk = torch.topk(token_counts, k).indices.tolist()

    freqs = token_counts[topk].tolist()
    freqs = [int(round(x / sum(freqs) * total)) for x in freqs]

    print("Encoding: Top-k indices:", topk)
    print("Encoding: Frequencies (first 10):", freqs[:10], "Sum =", sum(freqs))
    return topk, freqs


def decode_text_topk(topk, model, tokenizer, total=4096, k=200, temperature=1.0, max_new_tokens=50):
    # Convert topk into a probability distribution
    logits = torch.zeros(model.config.vocab_size).to(model.device)
    logits[topk] = 1.0
    probs = torch.softmax(logits / temperature, dim=-1)

    input_ids = torch.tensor([[tokenizer.bos_token_id]]).to(model.device)
    generated = model.generate(
        input_ids,
        do_sample=True,
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.eos_token_id,
        temperature=temperature,
        top_k=0,
        top_p=1.0,
        logits_processor=[
            lambda input_ids, scores: scores.masked_fill(
                torch.tensor([i not in topk for i in range(scores.shape[-1])], device=scores.device), float('-inf')
            )
        ]
    )

    decoded = tokenizer.decode(generated[0], skip_special_tokens=True)
    return decoded


In [ ]:
sample_text = "<s>[INST] What is the capital of France? [/INST]"

print("\n--- Mistral Pipeline (top-k) ---")
topk_indices, freqs = encode_text_topk(sample_text, model, tokenizer, total=4096, k=200)
decoded_mistral = decode_text_topk(topk_indices, model, tokenizer, total=4096, k=200)
print("Decoded text (Mistral):\n", decoded_mistral)



--- Mistral Pipeline (top-k) ---
Encoding: Top-k indices: [415, 3916, 13, 1186, 302, 28804, 22478, 349, 272, 28705, 1014, 422, 460, 387, 23317, 28741, 2820, 1679, 1192, 318, 1247, 330, 1824, 5465, 401, 28725, 4008, 12567, 304, 365, 4843, 320, 7881, 382, 277, 2990, 28708, 1676, 384, 334, 28747, 399, 3169, 315, 351, 264, 13250, 9932, 3095, 16424, 17403, 19391, 325, 5558, 6664, 3298, 393, 367, 26486, 9903, 413, 10524, 7930, 8577, 10242, 3658, 4380, 22341, 2642, 3660, 5133, 2429, 4682, 1602, 2347, 3195, 500, 25355, 3435, 5077, 295, 420, 319, 1054, 20438, 28230, 261, 394, 284, 10828, 15171, 10036, 11076, 17812, 418, 726, 15341, 446, 7497, 4583, 7293, 907, 3945, 2693, 287, 2480, 451, 2093, 4192, 19311, 28740, 363, 6082, 275, 1015, 22390, 733, 305, 11482, 8007, 560, 1318, 281, 4720, 524, 24115, 10529, 23938, 307, 290, 1450, 268, 8964, 408, 28723, 5328, 1313, 25129, 285, 574, 3799, 461, 1235, 8433, 16160, 511, 19219, 613, 14642, 332, 550, 475, 4515, 23561, 686, 289, 3964, 851, 19652, 7687, 34

In [ ]:
def chat_with_mistral(prompt, max_new_tokens=100):
    # Instruction format
    formatted = f"<s>[INST] {prompt.strip()} [/INST]"

    inputs = tokenizer(formatted, return_tensors="pt").to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    print(decoded)

# Try this
chat_with_mistral("What is the capital of France? Answer with only 1 word")

[INST] What is the capital of France? Answer with only 1 word [/INST] Paris


In [ ]:
# Function to compute total compression in bits
def compute_bits_per_token(text):
    input_ids = tokenizer(text, return_tensors="pt").input_ids[0].to(model.device)

    total_bits = 0.0
    for i in range(len(input_ids) - 1):
        prefix = input_ids[:i+1].unsqueeze(0)
        target_token = input_ids[i+1]

        with torch.no_grad():
            logits = model(prefix).logits[0, -1]
            probs = torch.softmax(logits, dim=-1)
            prob = probs[target_token].item()

        bits = -torch.log2(torch.tensor(prob + 1e-12)).item()
        total_bits += bits

    bits_per_token = total_bits / (len(input_ids) - 1)
    return bits_per_token, total_bits, len(input_ids) - 1

# Example
text = "The capital of France is Paris."
bpt, total, n_tokens = compute_bits_per_token(text)
print(f"Text: {text}")
print(f"Tokens: {n_tokens}")
print(f"Total bits: {total:.2f}")
print(f"Bits per token: {bpt:.4f}")


Text: The capital of France is Paris.
Tokens: 7
Total bits: 35.49
Bits per token: 5.0697


# MIGHT CHANGE PROJECT TO ONLY FOCUS ON ENCODING AND NOT DECODING BECAUSE DECODING AND VERIFYING OUTPUTS IS BEING VERY ANNOYING.

---



But going to try the original encoding and decoding approach with vanilla Mistral quantized before I make this decision

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_4bit=True,
    trust_remote_code=True
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
import torch

def encode_text_topk(text, model, tokenizer, total=4096, k=200):
    input_ids = tokenizer(text, return_tensors="pt").input_ids.to(model.device)
    with torch.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits[0]  # Shape: (seq_len, vocab_size)

    probs = torch.softmax(logits, dim=-1)
    token_counts = torch.sum(probs, dim=0).cpu()
    topk = torch.topk(token_counts, k).indices.tolist()

    freqs = token_counts[topk].tolist()
    freqs = [int(round(x / sum(freqs) * total)) for x in freqs]

    print("Encoding: Top-k indices:", topk)
    print("Encoding: Frequencies (first 10):", freqs[:10], "Sum =", sum(freqs))
    return topk, freqs


def decode_text_topk(topk, model, tokenizer, total=4096, k=200, temperature=1.0, max_new_tokens=50):
    logits = torch.zeros(model.config.vocab_size).to(model.device)
    logits[topk] = 1.0
    probs = torch.softmax(logits / temperature, dim=-1)

    input_ids = torch.tensor([[tokenizer.bos_token_id]]).to(model.device)

    generated = model.generate(
        input_ids,
        do_sample=True,
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.eos_token_id,
        temperature=temperature,
        top_k=0,
        top_p=1.0,
        logits_processor=[
            lambda input_ids, scores: scores.masked_fill(
                torch.tensor([i not in topk for i in range(scores.shape[-1])], device=scores.device), float('-inf')
            )
        ]
    )

    decoded = tokenizer.decode(generated[0], skip_special_tokens=True)
    return decoded


In [ ]:
sample_text = "What is the capital of France?"

print("\n--- Mistral Pipeline (top-k) ---")
topk_indices, freqs = encode_text_topk(sample_text, model, tokenizer, total=4096, k=200)
decoded_mistral = decode_text_topk(topk_indices, model, tokenizer, total=4096, k=200)
print("Decoded text (Mistral):\n", decoded_mistral)



--- Mistral Pipeline (top-k) ---


/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Encoding: Top-k indices: [28804, 302, 272, 422, 13, 349, 5465, 264, 2990, 415, 5133, 378, 28809, 531, 1489, 511, 774, 315, 460, 28705, 1824, 1047, 1080, 1235, 661, 574, 396, 330, 28742, 513, 28725, 1691, 907, 851, 5558, 1247, 304, 3660, 995, 4843, 726, 1550, 320, 6032, 682, 28747, 368, 6664, 10828, 816, 6082, 624, 981, 382, 560, 3658, 2870, 456, 384, 1450, 6881, 298, 334, 541, 500, 3905, 1873, 318, 403, 365, 3964, 478, 387, 1725, 351, 5783, 325, 393, 12567, 367, 297, 420, 413, 401, 8433, 1602, 7293, 4673, 1263, 1305, 524, 13188, 7826, 4822, 20438, 4720, 418, 5077, 764, 17812, 3437, 13250, 5746, 1015, 399, 2378, 1684, 23561, 4315, 2434, 23063, 1186, 4380, 6926, 22072, 10242, 1192, 1054, 2476, 3758, 6247, 17915, 1141, 1413, 15171, 5169, 1136, 1387, 24450, 2480, 14322, 829, 22491, 19652, 6526, 16654, 2969, 451, 10529, 9500, 5151, 4867, 7299, 28230, 6025, 11170, 15701, 19186, 19219, 1023, 622, 14642, 394, 11482, 22478, 25129, 550, 2774, 345, 26486, 5684, 369, 3945, 22390, 5016, 1984, 2191,

# OKAY YEAH, JUST STICK TO ENCODING. DECODING IS NOT WORKING AND IS WAY TOO MESSY. MAIN PROJECT IS JUST INVESTIGATING IF FINE-TUNED LLMS CAN ENCODE DOMAIN-SPECIFIC TEXT IN LESS BITS THAN VANILLA LLMS AS WELL AS TRADITIONAL ENCODING ALGORITHMS LIKE GZIP AND LZ78/LZW